In [2111]:
import pandas as pd
import numpy as np
import copy
from datetime import datetime
import pickle 
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.model_selection import RandomizedSearchCV, RepeatedStratifiedKFold, cross_val_score
from sklearn.metrics import mean_squared_error, roc_auc_score
import ubjson


from sklearn.model_selection import GridSearchCV
import gzip


import lightgbm as lgb

pd.options.display.max_columns = 200
pd.options.display.max_rows = 200

In [2112]:
fp_clients = '../data/clients_attributes.csv'
fp_sales = '../data/sales.csv'

fp_active_promos = '../data/active_promos.csv'
fp_executed_promos = '../data/executed_promos.csv'
fp_test = '../data/test.csv'

df_active_promos = pd.read_csv(fp_active_promos, encoding='ISO-8859-1', sep=',')
df_clients = pd.read_csv(fp_clients, encoding='ISO-8859-1', sep=',')
df_executed_promos = pd.read_csv(fp_executed_promos, encoding='ISO-8859-1', sep=',')
df_sales = pd.read_csv(fp_sales, encoding='ISO-8859-1', sep=',')
df_test = pd.read_csv(fp_test, encoding='ISO-8859-1', sep=',')

In [2113]:
# Crear periodo, como el mes de la mediana entre fecha_desde y fecha_hasta
df_active_promos['fecha_desde_dt'] \
  = pd.to_datetime(df_active_promos['Fecha_Desde'], format = '%Y-%m-%d')
df_active_promos['fecha_hasta_dt'] \
  = pd.to_datetime(df_active_promos['Fecha_Hasta'], format = '%Y-%m-%d')

df_active_promos['mid_date'] \
  = df_active_promos['fecha_desde_dt'] + \
    (df_active_promos['fecha_hasta_dt'] - df_active_promos['fecha_desde_dt']) /2

df_active_promos['periodo'] = df_active_promos['mid_date'].apply(lambda dt: dt.replace(day = 1))

df_active_promos['month_year'] = df_active_promos['periodo'].dt.to_period('M')
# Dropear columnas innecesarias
df_active_promos = df_active_promos.drop(['Fecha_Desde', 'Fecha_Hasta', 'fecha_desde_dt',
                                          'fecha_hasta_dt', 'mid_date', 'periodo'], axis = 1)
df_active_promos.head(1)

,CodigoDC,Marca,Cupo,Cliente,month_year
0,297601,29,9,3213,2018-08


In [2114]:
fechas_active_promos = list(df_active_promos['month_year'].unique())
fechas_active_promos

[Period('2018-08', 'M'),
 Period('2018-09', 'M'),
 Period('2018-10', 'M'),
 Period('2018-11', 'M'),
 Period('2018-12', 'M'),
 Period('2019-01', 'M'),
 Period('2019-02', 'M'),
 Period('2019-03', 'M'),
 Period('2019-04', 'M'),
 Period('2019-07', 'M'),
 Period('2019-05', 'M'),
 Period('2019-06', 'M'),
 Period('2019-08', 'M'),
 Period('2019-09', 'M')]

In [2115]:
# Quitare manualmente los active_promos de septiembre 2019 debido a que no hay match con executed_promos
#df_active_promos = df_active_promos[df_active_promos['month_year']!=fechas_active_promos[-1]]
df_active_promos = df_active_promos[df_active_promos['month_year']!= '201909']

In [2116]:
df_active_promos['month_year'].unique()

<PeriodArray>
['2018-08', '2018-09', '2018-10', '2018-11', '2018-12', '2019-01', '2019-02',
 '2019-03', '2019-04', '2019-07', '2019-05', '2019-06', '2019-08']
Length: 13, dtype: period[M]

In [2117]:
print(df_executed_promos.isna().sum())
# Agregar label a executed_promos
df_executed_promos['label'] = 1
display(df_executed_promos.head(1))

CodigoDC    0
Cliente     0
Marca       0
Cupo        0
dtype: int64


,CodigoDC,Cliente,Marca,Cupo,label
0,297601,8410,29,9,1


In [2118]:
print(f"Numero de registros en active promos: {len(df_active_promos)}")
df_m1 = pd.merge(
  df_active_promos,
  df_executed_promos,
  on = ['CodigoDC', 'Cliente', 'Marca', 'Cupo'],
  how = 'left')
print(f"Numero de registros en df_m1: {len(df_m1)}")

Numero de registros en active promos: 385802
Numero de registros en df_m1: 385802


In [2119]:
# Llenamos NaN con 0, 0 implica que el cliente no acepto la promocion
df_m1 = df_m1.fillna(0)

In [2120]:
fechas_m1 = list(df_m1['month_year'].unique())
fechas_m1.sort()
fechas_m1

[Period('2018-08', 'M'),
 Period('2018-09', 'M'),
 Period('2018-10', 'M'),
 Period('2018-11', 'M'),
 Period('2018-12', 'M'),
 Period('2019-01', 'M'),
 Period('2019-02', 'M'),
 Period('2019-03', 'M'),
 Period('2019-04', 'M'),
 Period('2019-05', 'M'),
 Period('2019-06', 'M'),
 Period('2019-07', 'M'),
 Period('2019-08', 'M')]

In [2121]:
# Agregamos informacion de clientes DIRECTA
df_clients.head(1)

,Cliente,FechaAltaCliente,Region,Gerencia,SubCanal,TipoPoblacion,Estrato,EF
0,1,2009-03-03,4,21,12,1,1,1


In [2122]:
df_m2 = pd.merge(
  df_m1, 
  df_clients[['Cliente', 'Region', 'Gerencia', 'SubCanal', 'TipoPoblacion', 'Estrato', 'EF']],
  on = 'Cliente',
  how = 'left'
)
print(df_m2.isna().sum())
display(df_m2.head(1))

CodigoDC         0
Marca            0
Cupo             0
Cliente          0
month_year       0
label            0
Region           0
Gerencia         0
SubCanal         0
TipoPoblacion    0
Estrato          0
EF               0
dtype: int64


,CodigoDC,Marca,Cupo,Cliente,month_year,label,Region,Gerencia,SubCanal,TipoPoblacion,Estrato,EF
0,297601,29,9,3213,2018-08,0.0,3,8,15,2,3,1


In [2123]:
# Cargamos pickle
f_ventas = '../tablones/data_grupo_3.pkl'
def load_pickle(fp):
  with open(fp, 'rb') as f:
    df = pickle.load(f)
  return df
df_ventas_enriched = load_pickle(f_ventas)
print(df_ventas_enriched.isna().sum().sum())
# Creamos month_year
df_ventas_enriched['month_year'] = df_ventas_enriched['periodo'].dt.to_period('M')
df_ventas_enriched = df_ventas_enriched.drop('periodo', axis=1)
display(df_ventas_enriched.head(1))

0


,Cliente,Nr_sum,numero_ventas,Hl_sum,Dcto_sum,sum_2Nr,mean_2Nr,sum_2Hl,mean_2Hl,sum_2Dcto,mean_2Dcto,numero_ventas_sum,sum_2numero_ventas,mean_2numero_ventas,month_year
0,1,591975.69,13,2.31,-6785.54,680049.570526,340024.785263,2.491579,1.245789,-47691.214211,-23845.607105,13,17.736842,8.868421,2018-01


In [2124]:
# Agregamos data de ventas enriquecida
df_m3 = pd.merge(
  df_m2,
  df_ventas_enriched,
  on = ['month_year', 'Cliente'],
  how = 'left'
)
print(df_m3.isna().sum()/len(df_m3)*100)
display(df_m3.head(1))

CodigoDC               0.0
Marca                  0.0
Cupo                   0.0
Cliente                0.0
month_year             0.0
label                  0.0
Region                 0.0
Gerencia               0.0
SubCanal               0.0
TipoPoblacion          0.0
Estrato                0.0
EF                     0.0
Nr_sum                 0.0
numero_ventas          0.0
Hl_sum                 0.0
Dcto_sum               0.0
sum_2Nr                0.0
mean_2Nr               0.0
sum_2Hl                0.0
mean_2Hl               0.0
sum_2Dcto              0.0
mean_2Dcto             0.0
numero_ventas_sum      0.0
sum_2numero_ventas     0.0
mean_2numero_ventas    0.0
dtype: float64


,CodigoDC,Marca,Cupo,Cliente,month_year,label,Region,Gerencia,SubCanal,TipoPoblacion,Estrato,EF,Nr_sum,numero_ventas,Hl_sum,Dcto_sum,sum_2Nr,mean_2Nr,sum_2Hl,mean_2Hl,sum_2Dcto,mean_2Dcto,numero_ventas_sum,sum_2numero_ventas,mean_2numero_ventas
0,297601,29,9,3213,2018-08,0.0,3,8,15,2,3,1,398013.65,16,1.36,-32036.23,839393.42,419696.71,2.75,1.375,-44248.91,-22124.455,16,32.0,16.0


In [2125]:
# Cargar data de prom ofrecidas
# Arreglar fecha para poder cruzar sin problemas
# Quitar registros correspondientes a septiembre 2019
# Conversion de tipos para poder cruzar
f_promo_ofrecidas = '../tablones/num_prom_ofr.pkl'
def load_pickle(fp):
  with open(fp, 'rb') as f:
    df = pickle.load(f)
  return df
df_promo_ofrecidas = load_pickle(f_promo_ofrecidas)
df_promo_ofrecidas['Date_Desde'] \
  = pd.to_datetime(df_promo_ofrecidas['Date_Desde'], format= "%Y-%m")
# Convertir Date_Desde a periodo -> month_year
df_promo_ofrecidas['month_year'] = df_promo_ofrecidas['Date_Desde'].dt.to_period('M')
# Dropear columnas innecesarias
df_promo_ofrecidas = df_promo_ofrecidas.drop('Date_Desde', axis = 1)
# QUITAR DATA DE SEPTIEMBRE PORQUE ESTO NO SE USA PARA ENTRENAR POR VARIAS RAZONES
df_promo_ofrecidas = df_promo_ofrecidas[df_promo_ofrecidas['month_year'] != '201909']
# Convertir columnas a int para poder cruzar
df_promo_ofrecidas['Marca'] = df_promo_ofrecidas['Marca'].astype(int)
df_promo_ofrecidas['Cupo'] = df_promo_ofrecidas['Cupo'].astype(int)
df_promo_ofrecidas['Cliente'] = df_promo_ofrecidas['Cliente'].astype(int)

df_promo_ofrecidas.head(1)

,Marca,Cupo,Cliente,Num_prom_ofr,month_year
0,29,9,10358,1,2018-08


In [2126]:
# Cruzamos df_promo_ofrecidas(enriched) con df_m4
df_m4 = pd.merge(
  df_m3,
  df_promo_ofrecidas,
  on = ['month_year', 'Cliente', 'Marca', 'Cupo'],
  how = 'left'
)
print(df_m4.isna().sum()/len(df_m4)*100)
print(df_m4.isna().sum().sum())
display(df_m4.head(1))

CodigoDC               0.000000
Marca                  0.000000
Cupo                   0.000000
Cliente                0.000000
month_year             0.000000
label                  0.000000
Region                 0.000000
Gerencia               0.000000
SubCanal               0.000000
TipoPoblacion          0.000000
Estrato                0.000000
EF                     0.000000
Nr_sum                 0.000000
numero_ventas          0.000000
Hl_sum                 0.000000
Dcto_sum               0.000000
sum_2Nr                0.000000
mean_2Nr               0.000000
sum_2Hl                0.000000
mean_2Hl               0.000000
sum_2Dcto              0.000000
mean_2Dcto             0.000000
numero_ventas_sum      0.000000
sum_2numero_ventas     0.000000
mean_2numero_ventas    0.000000
Num_prom_ofr           0.415239
dtype: float64
1602


,CodigoDC,Marca,Cupo,Cliente,month_year,label,Region,Gerencia,SubCanal,TipoPoblacion,Estrato,EF,Nr_sum,numero_ventas,Hl_sum,Dcto_sum,sum_2Nr,mean_2Nr,sum_2Hl,mean_2Hl,sum_2Dcto,mean_2Dcto,numero_ventas_sum,sum_2numero_ventas,mean_2numero_ventas,Num_prom_ofr
0,297601,29,9,3213,2018-08,0.0,3,8,15,2,3,1,398013.65,16,1.36,-32036.23,839393.42,419696.71,2.75,1.375,-44248.91,-22124.455,16,32.0,16.0,1.0


In [2127]:
# Llenamos los nan de esta columna con 0 porque significa que no se les ha ofrecido productos para esa
# combinacion de marca, cupo, fecha, cliente
df_m4['Num_prom_ofr'] = df_m4['Num_prom_ofr'].fillna(0)
print(df_m4.isna().sum()[df_m4.isna().sum() > 0])
display(df_m4.head(1))

Series([], dtype: int64)


,CodigoDC,Marca,Cupo,Cliente,month_year,label,Region,Gerencia,SubCanal,TipoPoblacion,Estrato,EF,Nr_sum,numero_ventas,Hl_sum,Dcto_sum,sum_2Nr,mean_2Nr,sum_2Hl,mean_2Hl,sum_2Dcto,mean_2Dcto,numero_ventas_sum,sum_2numero_ventas,mean_2numero_ventas,Num_prom_ofr
0,297601,29,9,3213,2018-08,0.0,3,8,15,2,3,1,398013.65,16,1.36,-32036.23,839393.42,419696.71,2.75,1.375,-44248.91,-22124.455,16,32.0,16.0,1.0


In [2128]:
df_m4.shape

(385802, 26)

In [2129]:
f_prop = '../tablones/data_grupo2_proporcion.pkl' 
df_prop_enriched = load_pickle(f_prop)
df_prop_enriched.head(1)

,month_year,Cliente,ofertas_dadas
0,2018-08,1,0


In [2130]:
# Cruzamos df_prop_enriched(enriched) 
df_m5 = pd.merge(
  df_m4,
  df_prop_enriched,
  on = ['month_year', 'Cliente'],
  how = 'left'
)
print(df_m5.isna().sum()/len(df_m5)*100)
print(df_m5.isna().sum().sum())
display(df_m5.head(1))

CodigoDC               0.0
Marca                  0.0
Cupo                   0.0
Cliente                0.0
month_year             0.0
label                  0.0
Region                 0.0
Gerencia               0.0
SubCanal               0.0
TipoPoblacion          0.0
Estrato                0.0
EF                     0.0
Nr_sum                 0.0
numero_ventas          0.0
Hl_sum                 0.0
Dcto_sum               0.0
sum_2Nr                0.0
mean_2Nr               0.0
sum_2Hl                0.0
mean_2Hl               0.0
sum_2Dcto              0.0
mean_2Dcto             0.0
numero_ventas_sum      0.0
sum_2numero_ventas     0.0
mean_2numero_ventas    0.0
Num_prom_ofr           0.0
ofertas_dadas          0.0
dtype: float64
0


,CodigoDC,Marca,Cupo,Cliente,month_year,label,Region,Gerencia,SubCanal,TipoPoblacion,Estrato,EF,Nr_sum,numero_ventas,Hl_sum,Dcto_sum,sum_2Nr,mean_2Nr,sum_2Hl,mean_2Hl,sum_2Dcto,mean_2Dcto,numero_ventas_sum,sum_2numero_ventas,mean_2numero_ventas,Num_prom_ofr,ofertas_dadas
0,297601,29,9,3213,2018-08,0.0,3,8,15,2,3,1,398013.65,16,1.36,-32036.23,839393.42,419696.71,2.75,1.375,-44248.91,-22124.455,16,32.0,16.0,1.0,1


In [2131]:
df_m5['Cliente'].value_counts()

5226     61
581      61
553      59
4138     59
9263     59
         ..
13724     1
14036     1
7650      1
11596     1
6131      1
Name: Cliente, Length: 13549, dtype: int64

In [2132]:
df_m6 = copy.deepcopy(df_m5)
df_m6.head(1)

,CodigoDC,Marca,Cupo,Cliente,month_year,label,Region,Gerencia,SubCanal,TipoPoblacion,Estrato,EF,Nr_sum,numero_ventas,Hl_sum,Dcto_sum,sum_2Nr,mean_2Nr,sum_2Hl,mean_2Hl,sum_2Dcto,mean_2Dcto,numero_ventas_sum,sum_2numero_ventas,mean_2numero_ventas,Num_prom_ofr,ofertas_dadas
0,297601,29,9,3213,2018-08,0.0,3,8,15,2,3,1,398013.65,16,1.36,-32036.23,839393.42,419696.71,2.75,1.375,-44248.91,-22124.455,16,32.0,16.0,1.0,1


In [2133]:
# Agregarmos 3 extra ventas_mc_new_1, ventas_extra_1, ventas_extra_2
df_extra_1 = load_pickle('../tablones/ventas_mc_new_1.pkl')
df_extra_2 = load_pickle('../tablones/ventas_extra_1.pkl')
df_extra_3 = load_pickle('../tablones/ventas_extra_2.pkl')

In [2134]:
df_extra_1.head(1)

,Marca,Cupo,month_year,Nr_MCsum,Nr_MCmean,Hl_MCsum,Hl_MCmean,Dcto_MCsum,Dcto_MCmean
0,1,30.0,2019-08,176028.75,3826.711957,1.51,0.032826,-57971.25,-1260.244565


In [2135]:
df_extra_2.head(1)

ClaseEnvase,month_year,Marca,Cupo,ClaseEnvase_1_sales,ClaseEnvase2_sales
0,2018-01,2,5.0,0,1


In [2136]:
df_extra_3.head(1)

SegmentoPrecio,month_year,Marca,Cupo,Segmento_1_sales,Segmento_2_sales,Segmento_3_sales,Segmento_5_sales
0,2018-01,2,5.0,1,0,0,0


In [2137]:
df_m7 = pd.merge(
  df_m6,
  df_extra_1,
  how='left',
  on = ['Marca', 'Cupo', 'month_year']
)
print(df_m7.isna().sum().sum())
display(df_m7.head(1))

0


,CodigoDC,Marca,Cupo,Cliente,month_year,label,Region,Gerencia,SubCanal,TipoPoblacion,Estrato,EF,Nr_sum,numero_ventas,Hl_sum,Dcto_sum,sum_2Nr,mean_2Nr,sum_2Hl,mean_2Hl,sum_2Dcto,mean_2Dcto,numero_ventas_sum,sum_2numero_ventas,mean_2numero_ventas,Num_prom_ofr,ofertas_dadas,Nr_MCsum,Nr_MCmean,Hl_MCsum,Hl_MCmean,Dcto_MCsum,Dcto_MCmean
0,297601,29,9,3213,2018-08,0.0,3,8,15,2,3,1,398013.65,16,1.36,-32036.23,839393.42,419696.71,2.75,1.375,-44248.91,-22124.455,16,32.0,16.0,1.0,1,50196409.28,47579.534863,77.78,0.073725,-4821672.84,-4570.306009


In [2138]:
df_m8 = pd.merge(
  df_m7,
  df_extra_2,
  how='left',
  on = ['Marca', 'Cupo', 'month_year']
)
print(df_m8.isna().sum().sum())
display(df_m8.head(1))

0


,CodigoDC,Marca,Cupo,Cliente,month_year,label,Region,Gerencia,SubCanal,TipoPoblacion,Estrato,EF,Nr_sum,numero_ventas,Hl_sum,Dcto_sum,sum_2Nr,mean_2Nr,sum_2Hl,mean_2Hl,sum_2Dcto,mean_2Dcto,numero_ventas_sum,sum_2numero_ventas,mean_2numero_ventas,Num_prom_ofr,ofertas_dadas,Nr_MCsum,Nr_MCmean,Hl_MCsum,Hl_MCmean,Dcto_MCsum,Dcto_MCmean,ClaseEnvase_1_sales,ClaseEnvase2_sales
0,297601,29,9,3213,2018-08,0.0,3,8,15,2,3,1,398013.65,16,1.36,-32036.23,839393.42,419696.71,2.75,1.375,-44248.91,-22124.455,16,32.0,16.0,1.0,1,50196409.28,47579.534863,77.78,0.073725,-4821672.84,-4570.306009,1055,0


In [2139]:
df_m9 = pd.merge(
  df_m8,
  df_extra_3,
  how='left',
  on = ['Marca', 'Cupo', 'month_year']
)
print(df_m9.isna().sum().sum())
display(df_m9.head(1))

0


,CodigoDC,Marca,Cupo,Cliente,month_year,label,Region,Gerencia,SubCanal,TipoPoblacion,Estrato,EF,Nr_sum,numero_ventas,Hl_sum,Dcto_sum,sum_2Nr,mean_2Nr,sum_2Hl,mean_2Hl,sum_2Dcto,mean_2Dcto,numero_ventas_sum,sum_2numero_ventas,mean_2numero_ventas,Num_prom_ofr,ofertas_dadas,Nr_MCsum,Nr_MCmean,Hl_MCsum,Hl_MCmean,Dcto_MCsum,Dcto_MCmean,ClaseEnvase_1_sales,ClaseEnvase2_sales,Segmento_1_sales,Segmento_2_sales,Segmento_3_sales,Segmento_5_sales
0,297601,29,9,3213,2018-08,0.0,3,8,15,2,3,1,398013.65,16,1.36,-32036.23,839393.42,419696.71,2.75,1.375,-44248.91,-22124.455,16,32.0,16.0,1.0,1,50196409.28,47579.534863,77.78,0.073725,-4821672.84,-4570.306009,1055,0,0,0,0,1055


In [2140]:
# Num_prom_ofr, para probar
df_m9 = df_m9.drop(['month_year', 'CodigoDC', 'Cliente', ], axis=1)
df_m9.head(1)

,Marca,Cupo,label,Region,Gerencia,SubCanal,TipoPoblacion,Estrato,EF,Nr_sum,numero_ventas,Hl_sum,Dcto_sum,sum_2Nr,mean_2Nr,sum_2Hl,mean_2Hl,sum_2Dcto,mean_2Dcto,numero_ventas_sum,sum_2numero_ventas,mean_2numero_ventas,Num_prom_ofr,ofertas_dadas,Nr_MCsum,Nr_MCmean,Hl_MCsum,Hl_MCmean,Dcto_MCsum,Dcto_MCmean,ClaseEnvase_1_sales,ClaseEnvase2_sales,Segmento_1_sales,Segmento_2_sales,Segmento_3_sales,Segmento_5_sales
0,29,9,0.0,3,8,15,2,3,1,398013.65,16,1.36,-32036.23,839393.42,419696.71,2.75,1.375,-44248.91,-22124.455,16,32.0,16.0,1.0,1,50196409.28,47579.534863,77.78,0.073725,-4821672.84,-4570.306009,1055,0,0,0,0,1055


In [2141]:
df_m9.to_parquet('../tablones/train_data_num_mini_v9_sin_drop_duplimes_prop.parquet', engine='pyarrow')

# DESDE ACA SE TRABAJA ASUMIENDO VARIABLES NUMERICAS

In [2142]:
df_m9.head(1)

,Marca,Cupo,label,Region,Gerencia,SubCanal,TipoPoblacion,Estrato,EF,Nr_sum,numero_ventas,Hl_sum,Dcto_sum,sum_2Nr,mean_2Nr,sum_2Hl,mean_2Hl,sum_2Dcto,mean_2Dcto,numero_ventas_sum,sum_2numero_ventas,mean_2numero_ventas,Num_prom_ofr,ofertas_dadas,Nr_MCsum,Nr_MCmean,Hl_MCsum,Hl_MCmean,Dcto_MCsum,Dcto_MCmean,ClaseEnvase_1_sales,ClaseEnvase2_sales,Segmento_1_sales,Segmento_2_sales,Segmento_3_sales,Segmento_5_sales
0,29,9,0.0,3,8,15,2,3,1,398013.65,16,1.36,-32036.23,839393.42,419696.71,2.75,1.375,-44248.91,-22124.455,16,32.0,16.0,1.0,1,50196409.28,47579.534863,77.78,0.073725,-4821672.84,-4570.306009,1055,0,0,0,0,1055


In [2143]:
df_m9.shape

(385802, 36)

In [2144]:
X_train_todo, X_test_todo, y_train_todo, y_test_todo \
  = train_test_split( df_m9.drop(["label"], axis=1), df_m9["label"], test_size = 0.2)

# LGBM

In [2145]:
opt_params = {'colsample_bytree': 0.6706735412200329, 
              'learning_rate ': 0.01,
              'max_depth': 100, 
              'min_child_samples': 19,
              'min_child_weight': 10.0,
              'n_estimators': 2000, 
              'num_leaves': 50, 
              'objective': 'binary', 
              'reg_alpha': 0.03, 
              'reg_lambda': 8, 
              'scale_pos_weight': 8,
              'subsample': 0.8488754927718809} 

classifier_lgbm_todo = lgb.LGBMClassifier(
  **opt_params
  

  
  
  
#   n_estimators=1000,
#   reg_lambda=10,
#   max_depth=-1,
#   min_child_samples = 20,
#   reg_alpha=0.05,
#   objective='binary',
#   scale_pos_weight = 8,

)

# params = {"early_stopping_rounds":20, 
#             "eval_metric" : 'auc', 
#             "eval_set" : [(X_test_todo, y_test_todo)],
#             'eval_names': ['valid'],
#             'verbose': False,
#             'categorical_feature': 'auto'}

classifier_lgbm_todo.fit(X_train_todo, y_train_todo) #**params
# make predictions for test data    
predictions = classifier_lgbm_todo.predict_proba(X_test_todo)[:, 1]
# evaluate predictions
print('ROC AUC Score',roc_auc_score(y_test_todo, predictions))
# old ROC AUC Score 0.8972606476313741

[LightGBM] [Warning] learning_rate is set=0.1, learning_rate= will be ignored. Current value: learning_rate=0.1
[LightGBM] [Warning] Unknown parameter: 0.01
ROC AUC Score 0.8878235399433733


In [2146]:
classifier_lgbm_todo.get_params

<bound method LGBMModel.get_params of LGBMClassifier(colsample_bytree=0.6706735412200329, learning_rate =0.01,
               max_depth=100, min_child_samples=19, min_child_weight=10.0,
               n_estimators=2000, num_leaves=50, objective='binary',
               reg_alpha=0.03, reg_lambda=8, scale_pos_weight=8,
               subsample=0.8488754927718809)>

# ARMAR EL TEST SET

In [2147]:
df_test = pd.read_csv(fp_test, encoding='ISO-8859-1', sep=',')
df_test.head(1)

,Cliente,Marca,Cupo,Ejecuto_Promo
0,10,40,16,NaN


In [2148]:
# Agregar month_year = '201908', se asume
df_test['month_year'] = pd.Period('2019-08', 'M')
# Dropear Ejecuto_Promo, al final lo agrego otra vez
df_test = df_test.drop('Ejecuto_Promo', axis = 1)
print(df_test.isna().sum().sum())
display(df_test.head(1))

0


,Cliente,Marca,Cupo,month_year
0,10,40,16,2019-08


In [2149]:
# Agregamos informacion de clientes sin enriquecer
df_t1 = pd.merge(
  df_test,
  df_clients[['Cliente', 'Region', 'Gerencia', 'SubCanal', 'TipoPoblacion', 'Estrato', 'EF']],
  on = 'Cliente',
  how = 'left'
)
print(df_t1.isna().sum().sum())
display(df_t1.head(1))

0


,Cliente,Marca,Cupo,month_year,Region,Gerencia,SubCanal,TipoPoblacion,Estrato,EF
0,10,40,16,2019-08,4,21,15,2,2,0


In [2150]:
# Agregamos informacion de ventas enriquecida
df_t2 = pd.merge(
  df_t1,
  df_ventas_enriched,
  on = ['month_year', 'Cliente'],
  how = 'left'  
)
print(df_t2.isna().sum().sum())
display(df_t2.head(1))

0


,Cliente,Marca,Cupo,month_year,Region,Gerencia,SubCanal,TipoPoblacion,Estrato,EF,Nr_sum,numero_ventas,Hl_sum,Dcto_sum,sum_2Nr,mean_2Nr,sum_2Hl,mean_2Hl,sum_2Dcto,mean_2Dcto,numero_ventas_sum,sum_2numero_ventas,mean_2numero_ventas
0,10,40,16,2019-08,4,21,15,2,2,0,818199.93,14,3.58,-43358.46,1521431.02,760715.51,6.66,3.33,-62775.26,-31387.63,14,29.0,14.5


In [2151]:
# ESTO SE DEBERIA IMPORTAR DE UN UTILITARIO
def obtener_prop_num(df, col_variable):
  """
  df: pd.Dataframe
  col_variable: str
  """
  # Copia: para asegurarnos que no modificamos df -> se debe hacer refactoring
  c_df = copy.deepcopy(df)
  # Cuenta de registros en tabla de clientes
  numero_clientes = len(c_df)
  
  # Contamos registros por valor de col_variable
  df_g1 = c_df.groupby([col_variable]).agg({
    'FechaAltaCliente': 'count',
  }).reset_index()
  
  # Creamos nombre de la columna donde se almacena el numero de clientes
  col_num_clientes = 'num_clientes_' + col_variable
  # Renombrar columna donde se almacena el numero de clientes
  df_g1 = df_g1.rename(columns={'FechaAltaCliente': col_num_clientes})
  # Creamos nombre de la columna donde se almacena la proporcion de clientes
  col_proporcion_clientes = 'prop_clientes_' + col_variable
  # Agregamos columna que almacena informacion de propocion de clientes 
  df_g1[col_proporcion_clientes] = df_g1[col_num_clientes] / numero_clientes
  
  return df_g1


# Agregamos informacion de prop_num 
cols_cat_clientes = ['Region', 'Gerencia', 'SubCanal', 'TipoPoblacion', 'Estrato', 'EF']
df_t3 = copy.deepcopy(df_t2)
for col_name in cols_cat_clientes:
  curr_df = obtener_prop_num(df_clients, col_name) 
  df_t3 = pd.merge(df_t3, curr_df, how = 'left', on = col_name)
  #print(df_t3.isna().sum().sum())
print(df_t3.isna().sum().sum())
display(df_t3.head(1))  

0


,Cliente,Marca,Cupo,month_year,Region,Gerencia,SubCanal,TipoPoblacion,Estrato,EF,Nr_sum,numero_ventas,Hl_sum,Dcto_sum,sum_2Nr,mean_2Nr,sum_2Hl,mean_2Hl,sum_2Dcto,mean_2Dcto,numero_ventas_sum,sum_2numero_ventas,mean_2numero_ventas,num_clientes_Region,prop_clientes_Region,num_clientes_Gerencia,prop_clientes_Gerencia,num_clientes_SubCanal,prop_clientes_SubCanal,num_clientes_TipoPoblacion,prop_clientes_TipoPoblacion,num_clientes_Estrato,prop_clientes_Estrato,num_clientes_EF,prop_clientes_EF
0,10,40,16,2019-08,4,21,15,2,2,0,818199.93,14,3.58,-43358.46,1521431.02,760715.51,6.66,3.33,-62775.26,-31387.63,14,29.0,14.5,3531,0.234322,434,0.028801,2293,0.152167,11718,0.777623,6598,0.437853,9664,0.641317


In [2152]:
# Agregar info de promo ofrecidas
df_t4 = pd.merge(
  df_t3,
  df_promo_ofrecidas,
  on = ['month_year', 'Cliente', 'Marca', 'Cupo'],
  how = 'left'
)
print(df_t4.isna().sum().sum())
display(df_t4.head(1))  

1025


,Cliente,Marca,Cupo,month_year,Region,Gerencia,SubCanal,TipoPoblacion,Estrato,EF,Nr_sum,numero_ventas,Hl_sum,Dcto_sum,sum_2Nr,mean_2Nr,sum_2Hl,mean_2Hl,sum_2Dcto,mean_2Dcto,numero_ventas_sum,sum_2numero_ventas,mean_2numero_ventas,num_clientes_Region,prop_clientes_Region,num_clientes_Gerencia,prop_clientes_Gerencia,num_clientes_SubCanal,prop_clientes_SubCanal,num_clientes_TipoPoblacion,prop_clientes_TipoPoblacion,num_clientes_Estrato,prop_clientes_Estrato,num_clientes_EF,prop_clientes_EF,Num_prom_ofr
0,10,40,16,2019-08,4,21,15,2,2,0,818199.93,14,3.58,-43358.46,1521431.02,760715.51,6.66,3.33,-62775.26,-31387.63,14,29.0,14.5,3531,0.234322,434,0.028801,2293,0.152167,11718,0.777623,6598,0.437853,9664,0.641317,2.0


In [2153]:
# Llena con 0 los NaN en Num_prom_ofr porque significa que no se ofrecieron descuentos
df_t4['Num_prom_ofr'] = df_t4['Num_prom_ofr'].fillna(0)
print(df_t4.isna().sum().sum())
display(df_t4.head(1))  

0


,Cliente,Marca,Cupo,month_year,Region,Gerencia,SubCanal,TipoPoblacion,Estrato,EF,Nr_sum,numero_ventas,Hl_sum,Dcto_sum,sum_2Nr,mean_2Nr,sum_2Hl,mean_2Hl,sum_2Dcto,mean_2Dcto,numero_ventas_sum,sum_2numero_ventas,mean_2numero_ventas,num_clientes_Region,prop_clientes_Region,num_clientes_Gerencia,prop_clientes_Gerencia,num_clientes_SubCanal,prop_clientes_SubCanal,num_clientes_TipoPoblacion,prop_clientes_TipoPoblacion,num_clientes_Estrato,prop_clientes_Estrato,num_clientes_EF,prop_clientes_EF,Num_prom_ofr
0,10,40,16,2019-08,4,21,15,2,2,0,818199.93,14,3.58,-43358.46,1521431.02,760715.51,6.66,3.33,-62775.26,-31387.63,14,29.0,14.5,3531,0.234322,434,0.028801,2293,0.152167,11718,0.777623,6598,0.437853,9664,0.641317,2.0


In [2154]:
f_prop = '../tablones/data_grupo2_proporcion.pkl' 
df_prop_enriched = load_pickle(f_prop)
df_prop_enriched.head(1)

# Cruzamos df_prop_enriched(enriched) 
df_t5 = pd.merge(
  df_t4,
  df_prop_enriched,
  on = ['month_year', 'Cliente'],
  how = 'left'
)
print(df_t5.isna().sum()/len(df_t5)*100)
print(df_t5.isna().sum().sum())
display(df_t5.head(1))

Cliente                        0.0
Marca                          0.0
Cupo                           0.0
month_year                     0.0
Region                         0.0
Gerencia                       0.0
SubCanal                       0.0
TipoPoblacion                  0.0
Estrato                        0.0
EF                             0.0
Nr_sum                         0.0
numero_ventas                  0.0
Hl_sum                         0.0
Dcto_sum                       0.0
sum_2Nr                        0.0
mean_2Nr                       0.0
sum_2Hl                        0.0
mean_2Hl                       0.0
sum_2Dcto                      0.0
mean_2Dcto                     0.0
numero_ventas_sum              0.0
sum_2numero_ventas             0.0
mean_2numero_ventas            0.0
num_clientes_Region            0.0
prop_clientes_Region           0.0
num_clientes_Gerencia          0.0
prop_clientes_Gerencia         0.0
num_clientes_SubCanal          0.0
prop_clientes_SubCan

,Cliente,Marca,Cupo,month_year,Region,Gerencia,SubCanal,TipoPoblacion,Estrato,EF,Nr_sum,numero_ventas,Hl_sum,Dcto_sum,sum_2Nr,mean_2Nr,sum_2Hl,mean_2Hl,sum_2Dcto,mean_2Dcto,numero_ventas_sum,sum_2numero_ventas,mean_2numero_ventas,num_clientes_Region,prop_clientes_Region,num_clientes_Gerencia,prop_clientes_Gerencia,num_clientes_SubCanal,prop_clientes_SubCanal,num_clientes_TipoPoblacion,prop_clientes_TipoPoblacion,num_clientes_Estrato,prop_clientes_Estrato,num_clientes_EF,prop_clientes_EF,Num_prom_ofr,ofertas_dadas
0,10,40,16,2019-08,4,21,15,2,2,0,818199.93,14,3.58,-43358.46,1521431.02,760715.51,6.66,3.33,-62775.26,-31387.63,14,29.0,14.5,3531,0.234322,434,0.028801,2293,0.152167,11718,0.777623,6598,0.437853,9664,0.641317,2.0,5


In [2155]:
df_t5.shape

(16870, 37)

In [2156]:
df_t5.head(1)

,Cliente,Marca,Cupo,month_year,Region,Gerencia,SubCanal,TipoPoblacion,Estrato,EF,Nr_sum,numero_ventas,Hl_sum,Dcto_sum,sum_2Nr,mean_2Nr,sum_2Hl,mean_2Hl,sum_2Dcto,mean_2Dcto,numero_ventas_sum,sum_2numero_ventas,mean_2numero_ventas,num_clientes_Region,prop_clientes_Region,num_clientes_Gerencia,prop_clientes_Gerencia,num_clientes_SubCanal,prop_clientes_SubCanal,num_clientes_TipoPoblacion,prop_clientes_TipoPoblacion,num_clientes_Estrato,prop_clientes_Estrato,num_clientes_EF,prop_clientes_EF,Num_prom_ofr,ofertas_dadas
0,10,40,16,2019-08,4,21,15,2,2,0,818199.93,14,3.58,-43358.46,1521431.02,760715.51,6.66,3.33,-62775.26,-31387.63,14,29.0,14.5,3531,0.234322,434,0.028801,2293,0.152167,11718,0.777623,6598,0.437853,9664,0.641317,2.0,5


In [2157]:
# Creamos la columna de los 3 productos

In [2158]:
X_train_todo.head(1)

,Marca,Cupo,Region,Gerencia,SubCanal,TipoPoblacion,Estrato,EF,Nr_sum,numero_ventas,Hl_sum,Dcto_sum,sum_2Nr,mean_2Nr,sum_2Hl,mean_2Hl,sum_2Dcto,mean_2Dcto,numero_ventas_sum,sum_2numero_ventas,mean_2numero_ventas,Num_prom_ofr,ofertas_dadas,Nr_MCsum,Nr_MCmean,Hl_MCsum,Hl_MCmean,Dcto_MCsum,Dcto_MCmean,ClaseEnvase_1_sales,ClaseEnvase2_sales,Segmento_1_sales,Segmento_2_sales,Segmento_3_sales,Segmento_5_sales
126973,40,16,4,26,9,2,2,1,237692.36,8,1.05,-7440.22,467540.49,233770.245,2.01,1.005,-8846.94,-4423.47,8,16.0,8.0,3.0,6,1.297365e+09,209827.720839,4468.14,0.722649,-1.402767e+08,-22687.485672,0,6183,6183,0,0,0


In [2159]:
df_t5.head(1)

,Cliente,Marca,Cupo,month_year,Region,Gerencia,SubCanal,TipoPoblacion,Estrato,EF,Nr_sum,numero_ventas,Hl_sum,Dcto_sum,sum_2Nr,mean_2Nr,sum_2Hl,mean_2Hl,sum_2Dcto,mean_2Dcto,numero_ventas_sum,sum_2numero_ventas,mean_2numero_ventas,num_clientes_Region,prop_clientes_Region,num_clientes_Gerencia,prop_clientes_Gerencia,num_clientes_SubCanal,prop_clientes_SubCanal,num_clientes_TipoPoblacion,prop_clientes_TipoPoblacion,num_clientes_Estrato,prop_clientes_Estrato,num_clientes_EF,prop_clientes_EF,Num_prom_ofr,ofertas_dadas
0,10,40,16,2019-08,4,21,15,2,2,0,818199.93,14,3.58,-43358.46,1521431.02,760715.51,6.66,3.33,-62775.26,-31387.63,14,29.0,14.5,3531,0.234322,434,0.028801,2293,0.152167,11718,0.777623,6598,0.437853,9664,0.641317,2.0,5


In [2160]:
X_train_todo.columns

Index(['Marca', 'Cupo', 'Region', 'Gerencia', 'SubCanal', 'TipoPoblacion',
       'Estrato', 'EF', 'Nr_sum', 'numero_ventas', 'Hl_sum', 'Dcto_sum',
       'sum_2Nr', 'mean_2Nr', 'sum_2Hl', 'mean_2Hl', 'sum_2Dcto', 'mean_2Dcto',
       'numero_ventas_sum', 'sum_2numero_ventas', 'mean_2numero_ventas',
       'Num_prom_ofr', 'ofertas_dadas', 'Nr_MCsum', 'Nr_MCmean', 'Hl_MCsum',
       'Hl_MCmean', 'Dcto_MCsum', 'Dcto_MCmean', 'ClaseEnvase_1_sales',
       'ClaseEnvase2_sales', 'Segmento_1_sales', 'Segmento_2_sales',
       'Segmento_3_sales', 'Segmento_5_sales'],
      dtype='object')

In [2161]:
# Agregamos 3 tablas extras de ventas

In [2162]:
# Agregarmos 3 extra ventas_mc_new_1, ventas_extra_1, ventas_extra_2
df_extra_1 = load_pickle('../tablones/ventas_mc_new_1.pkl')
df_extra_2 = load_pickle('../tablones/ventas_extra_1.pkl')
df_extra_3 = load_pickle('../tablones/ventas_extra_2.pkl')

In [2163]:
df_extra_1.head(1)

,Marca,Cupo,month_year,Nr_MCsum,Nr_MCmean,Hl_MCsum,Hl_MCmean,Dcto_MCsum,Dcto_MCmean
0,1,30.0,2019-08,176028.75,3826.711957,1.51,0.032826,-57971.25,-1260.244565


In [2164]:
df_extra_2.head(1)

ClaseEnvase,month_year,Marca,Cupo,ClaseEnvase_1_sales,ClaseEnvase2_sales
0,2018-01,2,5.0,0,1


In [2165]:
df_extra_3.head(1)

SegmentoPrecio,month_year,Marca,Cupo,Segmento_1_sales,Segmento_2_sales,Segmento_3_sales,Segmento_5_sales
0,2018-01,2,5.0,1,0,0,0


In [2166]:
df_t6 = pd.merge(
  df_t5,
  df_extra_1,
  how='left',
  on = ['Marca', 'Cupo', 'month_year']
)
print(df_t6.isna().sum().sum())
display(df_t6.head(1))

0


,Cliente,Marca,Cupo,month_year,Region,Gerencia,SubCanal,TipoPoblacion,Estrato,EF,Nr_sum,numero_ventas,Hl_sum,Dcto_sum,sum_2Nr,mean_2Nr,sum_2Hl,mean_2Hl,sum_2Dcto,mean_2Dcto,numero_ventas_sum,sum_2numero_ventas,mean_2numero_ventas,num_clientes_Region,prop_clientes_Region,num_clientes_Gerencia,prop_clientes_Gerencia,num_clientes_SubCanal,prop_clientes_SubCanal,num_clientes_TipoPoblacion,prop_clientes_TipoPoblacion,num_clientes_Estrato,prop_clientes_Estrato,num_clientes_EF,prop_clientes_EF,Num_prom_ofr,ofertas_dadas,Nr_MCsum,Nr_MCmean,Hl_MCsum,Hl_MCmean,Dcto_MCsum,Dcto_MCmean
0,10,40,16,2019-08,4,21,15,2,2,0,818199.93,14,3.58,-43358.46,1521431.02,760715.51,6.66,3.33,-62775.26,-31387.63,14,29.0,14.5,3531,0.234322,434,0.028801,2293,0.152167,11718,0.777623,6598,0.437853,9664,0.641317,2.0,5,7.560536e+08,172851.762506,2581.13,0.590107,-36924213.72,-8441.749822


In [2167]:
df_t7 = pd.merge(
  df_t6,
  df_extra_2,
  how='left',
  on = ['Marca', 'Cupo', 'month_year']
)
print(df_t7.isna().sum().sum())
display(df_t7.head(1))

0


,Cliente,Marca,Cupo,month_year,Region,Gerencia,SubCanal,TipoPoblacion,Estrato,EF,Nr_sum,numero_ventas,Hl_sum,Dcto_sum,sum_2Nr,mean_2Nr,sum_2Hl,mean_2Hl,sum_2Dcto,mean_2Dcto,numero_ventas_sum,sum_2numero_ventas,mean_2numero_ventas,num_clientes_Region,prop_clientes_Region,num_clientes_Gerencia,prop_clientes_Gerencia,num_clientes_SubCanal,prop_clientes_SubCanal,num_clientes_TipoPoblacion,prop_clientes_TipoPoblacion,num_clientes_Estrato,prop_clientes_Estrato,num_clientes_EF,prop_clientes_EF,Num_prom_ofr,ofertas_dadas,Nr_MCsum,Nr_MCmean,Hl_MCsum,Hl_MCmean,Dcto_MCsum,Dcto_MCmean,ClaseEnvase_1_sales,ClaseEnvase2_sales
0,10,40,16,2019-08,4,21,15,2,2,0,818199.93,14,3.58,-43358.46,1521431.02,760715.51,6.66,3.33,-62775.26,-31387.63,14,29.0,14.5,3531,0.234322,434,0.028801,2293,0.152167,11718,0.777623,6598,0.437853,9664,0.641317,2.0,5,7.560536e+08,172851.762506,2581.13,0.590107,-36924213.72,-8441.749822,0,4374


In [2168]:
df_t8 = pd.merge(
  df_t7,
  df_extra_3,
  how='left',
  on = ['Marca', 'Cupo', 'month_year']
)
print(df_t8.isna().sum().sum())
display(df_t8.head(1))

0


,Cliente,Marca,Cupo,month_year,Region,Gerencia,SubCanal,TipoPoblacion,Estrato,EF,Nr_sum,numero_ventas,Hl_sum,Dcto_sum,sum_2Nr,mean_2Nr,sum_2Hl,mean_2Hl,sum_2Dcto,mean_2Dcto,numero_ventas_sum,sum_2numero_ventas,mean_2numero_ventas,num_clientes_Region,prop_clientes_Region,num_clientes_Gerencia,prop_clientes_Gerencia,num_clientes_SubCanal,prop_clientes_SubCanal,num_clientes_TipoPoblacion,prop_clientes_TipoPoblacion,num_clientes_Estrato,prop_clientes_Estrato,num_clientes_EF,prop_clientes_EF,Num_prom_ofr,ofertas_dadas,Nr_MCsum,Nr_MCmean,Hl_MCsum,Hl_MCmean,Dcto_MCsum,Dcto_MCmean,ClaseEnvase_1_sales,ClaseEnvase2_sales,Segmento_1_sales,Segmento_2_sales,Segmento_3_sales,Segmento_5_sales
0,10,40,16,2019-08,4,21,15,2,2,0,818199.93,14,3.58,-43358.46,1521431.02,760715.51,6.66,3.33,-62775.26,-31387.63,14,29.0,14.5,3531,0.234322,434,0.028801,2293,0.152167,11718,0.777623,6598,0.437853,9664,0.641317,2.0,5,7.560536e+08,172851.762506,2581.13,0.590107,-36924213.72,-8441.749822,0,4374,4374,0,0,0


# Hacemos que df_t5 tenga las mismas columnas que el X_train_todo

In [2169]:
df_t8 = df_t8[list(X_train_todo.columns)]

In [2170]:
df_t8.isna().sum().sum()

0

## Prediccion

In [2171]:
#pred = classifier_lgbm_dummies.predict_proba(df_t4_dummies)
#pred = classifier_prod.predict_proba(df_t5_dummies)
pred = classifier_lgbm_todo.predict_proba(df_t8)

In [2172]:
print(pred)

[[3.04854407e-01 6.95145593e-01]
 [9.57283981e-01 4.27160188e-02]
 [9.99993761e-01 6.23871027e-06]
 ...
 [8.05959944e-01 1.94040056e-01]
 [9.99876429e-01 1.23570876e-04]
 [9.76512191e-01 2.34878095e-02]]


In [2173]:
lista_pred_compro = [x for _, x in pred]
len(lista_pred_compro)

16870

In [2174]:
df_output = pd.read_csv(fp_test, encoding='ISO-8859-1', sep=',')
df_output.tail(1)

,Cliente,Marca,Cupo,Ejecuto_Promo
16869,15064,29,9,NaN


In [2175]:
df_output['Ejecuto_Promo'] = lista_pred_compro
print(df_output.isna().sum().sum())
display(df_output.head(1))

0


,Cliente,Marca,Cupo,Ejecuto_Promo
0,10,40,16,0.695146


In [2176]:
import time
fp = '../output/' +  str(time.asctime()) + '.csv'
fp = '../output/' + 'asddd.csv'
fp

'../output/asddd.csv'

In [2177]:
# Almacenar 
df_output = df_output.reset_index(drop=True)
display(df_output.head(1))


,Cliente,Marca,Cupo,Ejecuto_Promo
0,10,40,16,0.695146


In [2178]:
df_output.to_csv(fp, sep = ',', columns = ['Cliente', 'Marca', 'Cupo', 'Ejecuto_Promo'], index=False)